# Inference from a feature store

In [1]:
from mlops2_with_dagster.utils import get_project_dir, printse
from pathlib import Path
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /Users/rahul/Websites/mlops2_with_dagster


In [2]:
import pandas as pd
from joblib import dump, load
from hamilton import driver, base

In [6]:
model_file: str = f"{project_dir}/warehouse/rf.joblib"
features_file: str = f"{project_dir}/warehouse/featurestore_test.pkl"

In [7]:
# parameters
import dagstermill


inference_features = pd.read_pickle(features_file)
clfinfo = load(model_file)
context = dagstermill.get_context(op_config={'infertype': 'xxxx'})


In [ ]:
#infertype: str = context.op_config['infertype']

In [8]:
index_col = 'passengerid'
target_col = "survived"
config_model = {
    'index_column': index_col,
    'target_column': target_col,
    'random_state': 42,
    'max_depth': None,
    'validation_size_fraction': 0.33,
    't': 0.5
}

In [9]:
from mlops2_with_dagster import model_pipeline

## Inference pipeline 1: Inference from an already transformed data, or if you like, from a feature store

In [10]:
infer1_adapter = base.SimplePythonGraphAdapter(base.DictResult())
infer1_dr = driver.Driver(config_model, 
                            model_pipeline,
                            adapter = infer1_adapter)
dinfer1 = dict(
    X = inference_features,
    clf = clfinfo['fit_clf']
)

# infer1_dr.visualize_execution(['predictions'],
#                                        f"{project_dir}/artifacts/infer1_{infer_type}.dot", 
#                                        {}, 
#                                        inputs = dinfer1
# )

In [11]:
infer1dict = infer1_dr.execute(['predictions'],inputs = dinfer1)

In [12]:
#infer1dict['infertype'] = infertype

{'predictions': (array([0.03, 0.29, 0.82, ..., 0.  , 0.97, 0.83]),
  array([0, 0, 1, ..., 0, 1, 1]))}

In [15]:
# dump(infer1dict, f"{project_dir}/results/infer_{infertype}.joblib")

['/Users/rahul/Websites/mlops2_with_dagster/results/infer_test.joblib']

In [14]:
import dagstermill

dagstermill.yield_result(infer1dict, output_name="inference_results")

{'predictions': (array([0.03, 0.29, 0.82, ..., 0.  , 0.97, 0.83]),
  array([0, 0, 1, ..., 0, 1, 1]))}